<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%83%90%EC%83%89(4)%EC%BB%AC%EB%9F%BC_%EB%B6%84%EB%A5%98_210801.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **디스플레이 생산라인 센서 데이터 분석

## 주제 : 불량품에 영향을 주는 요인 찾기
- 결과 : "ㅇㅇ센서에서 ㅁㅁ면 불량품"
1. 주요 요인 피처 5개 추출
2. 불량품/양품 구분하는 모델도 찾아야 함


- 머신러닝 이용
---

## 컬럼이름 분류하기
- 데이터는 센서 데이터로 약  800여개의 컬럼이 있음
- 이름이 비슷한 컬럼이 다수 있으며 생산라인이 Left, Right으로 나뉘어 동일한 공정을 수행중임
- 컬럼이름을 기준으로 컬럼을 분류하고자 함
    - LEFT
    - RIGHT
    - 공통
- 컬럼을 분류하면 피처 수를 줄여 데이터탐색이 가능함
    - 이름이 유사한 컬럼간 종속성을 확인
    - LEFT, RIGHT 각 공정에 따라 레이블에 미치는 영향 확인 


In [1]:
# 데이터 불러오기
import pandas as pd
df_origin = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949')

In [2]:
# 컬럼이름 리스트 저장하기
col = df_origin.columns.tolist()
#col

### 1. 앞 3글자가 동일한 컬럼을 분류하고 확인


In [87]:
# 앞글자 3자리가 같은 컬럼의 집단 개수 확인
col_name_3=[]
for name in col:
    name = str(name)[:3]
    col_name_3.append(name)

print(len(set(col_name_3)))  # 61개

61


In [88]:
# 앞글자 3자리가 key, 이름이 유사한 컬럼이 value인 딕셔너리 생성
# 빈 딕셔너리 생성
col_name_3={}
# 인덱스, 컬럼이름에 대해 반복
for i, key in enumerate(col):
    key = str(key)[:3]
    # 만약 key가 딕셔너리에 없으면 키, 인덱스 생성
    if key not in col_name_3:
        col_name_3[key] = [col[i]]
        ## 딕셔너리에 넣을 때 int가 아닌 list로 넣어야 함

    # 만약 key가 딕셔너리에 있으면 인덱스 추가
    else:
        col_name_3[key].append(col[i])

print(len(col_name_3)) # 컬럼집단 61개 잘 만들어짐
#col_name_3

61


In [134]:
# 2. 컬럼명에 LEFT, RIGHT가 있으면 분류

# left, right 분류하여 저장할 딕셔너리 생성
dict_l_r = {'Left':[],
            'Right':[],
            'Center':[]}

# 각 key에 대해 반복 수행
for key in col_name_3:
    #print(col_name_3[key])
    #print(len(col_name_3[key]))


    for i in range(len(col_name_3[key])):
        #print(str(col_name_3[key][i]).lower())

        # left라는 단어가 있으면 
        if ('left') in str(col_name_3[key][i].lower()):
            dict_l_r['Left'].append(col_name_3[key][i])
        
        # right라는 단어가 있으면
        elif ('right') in str(col_name_3[key][i].lower()):
            dict_l_r['Right'].append(col_name_3[key][i])

        # left, right가 아니면
        #else:
        #    dict_l_r['Center'].append(col_name_3[key][i])

    # 각 컬럼이름에 대해 반복 수행
        #print(str(col_name_3[key]).lower())

# 확인
for key in dict_l_r:
    print(key, ":", len(dict_l_r[key]))
#dict_l_r

Left : 27
Right : 24
Center : 0


In [ ]:
# 3. 컬럼명에 _L, _R...등등이 있으면 분류
# 딕셔너리 개수 균형 확인해보면서 작업

for key in col_name_3:
    for i in range(len(col_name_3[key])):

        # 컬럼이름에 문자열 'ㅣ\_r\_', 'lr'가 포함된 경우 딕셔너리에 추가
        # re.search에서 여러개의 패턴을 or조건으로 잇기 위해 '' 안에서 |로 이어야 함
        re_isin_r_l = re.search('l\_r\_|lr', str(col_name_3[key][i].lower())) != None
        if re_isin_r_l:
            dict_l_r['Center'].append(col_name_3[key][i])

        else:
            # 컬럼이름에 문자열 '\_r\_'가 포함된 경우 딕셔너리에 추가
            re_isin_r = re.search('\_r\_|\.r\.|\_l\_r', str(col_name_3[key][i].lower())) != None
            if re_isin_r:
                dict_l_r['Right'].append(col_name_3[key][i])

            # 컬럼이름에 문자열 '\_l\_'가 포함된 경우 딕셔너리에 추가
            # l_r이 Left로 들어가는 것을 예방하기 위해 Right를 먼저 판단하고 Left 판단함 
            else:
                re_isin_l = re.search('\_l\_|\.l\.|\_l\_l', str(col_name_3[key][i].lower())) != None
                if re_isin_l:
                    dict_l_r['Left'].append(col_name_3[key][i])


# 확인
for key in dict_l_r:
    print(key, ":", len(dict_l_r[key]))
dict_l_r
#print(69+64+43)

In [136]:
##################33

# left, right 분류하여 저장할 딕셔너리 생성
dict_l_r = {'Left':[],
            'Right':[],
            'Center':[]}

# 3. 컬럼명에 _L, _R...등등이 있으면 분류
# 딕셔너리 개수 균형 확인해보면서 작업

for key in col_name_3:
    for i in range(len(col_name_3[key])):

        # 컬럼이름에 문자열 'ㅣ\_r\_', 'lr'가 포함된 경우 딕셔너리에 추가
        # re.search에서 여러개의 패턴을 or조건으로 잇기 위해 '' 안에서 |로 이어야 함
        re_isin_r_l = re.search('l\_r\_|lr', str(col_name_3[key][i].lower())) != None
        if re_isin_r_l:
            dict_l_r['Center'].append(col_name_3[key][i])

        else:
            # 컬럼이름에 문자열 '\_r\_'가 포함된 경우 딕셔너리에 추가
            re_isin_r = re.search('right|\_r\_|\.r\.|\_l\_r', str(col_name_3[key][i].lower())) != None
            if re_isin_r:
                dict_l_r['Right'].append(col_name_3[key][i])

            # 컬럼이름에 문자열 '\_l\_'가 포함된 경우 딕셔너리에 추가
            # l_r이 Left로 들어가는 것을 예방하기 위해 Right를 먼저 판단하고 Left 판단함 
            else:
                re_isin_l = re.search('left|\_l\_|\.l\.|\_l\_l', str(col_name_3[key][i].lower())) != None
                if re_isin_l:
                    dict_l_r['Left'].append(col_name_3[key][i])


# 확인
for key in dict_l_r:
    print(key, ":", len(dict_l_r[key]))
dict_l_r['Left']

Left : 67
Right : 64
Center : 43


['DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV',
 'DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV.1',
 'DB.HEAT.BTM.LEFT.1.TMP.1TI40315.PV',
 'DB.HEAT.BTM.LEFT.2.TMP.1TI40316.PV',
 'Left.edge',
 'TMP.TIN..BAY.1.LEFT.1TI30201.PV',
 'TMP.TIN..BAY.4.LEFT.1TI30203.PV',
 'TMP.TIN..BAY.5.LEFT.1TI30205.PV',
 'TMP.TIN..BAY.7.LEFT.1TI30207.PV',
 'TMP.TIN..BAY.10.LEFT.1TI30209.PV',
 'TMP.GLASS..LEFT.EXIT.1TIC30109.PV',
 'X.10.BAY.LEFT.PRESSURE.1CLBAY10LEFT_CPV',
 'X.1.BAY.LEFT.PRESSURE.1CLBAY1LEFT_CPV',
 'X.7.BAY.LEFT.PRESSURE.1CLBAY7LEFT_CPV',
 'RET.AMBIANT.LEFT.TMP.1TI42602.PV',
 'BATH.AMBIENT.3.BAY.LEFT.1TI31003_PV',
 'EXIT.LIP.PLATE.LEFT.1TI30604.PV',
 'E.L.DB.LEFT..N2.FLOW.1FI33501.PV',
 'DROSS.BOX.N2.BTM.HT1.L.1JI39005.PV',
 'DROSS.BOX.N2.BTM.HT..L..1JI39009.PV',
 'DROSSBOX.N2.SCN.L.1JI39011_PV',
 'HOOD.N2.TOP.HEATER.L..1JI39007.PV',
 'DDP.LEFT.N2.H2.9..1FI33420.PV',
 'SPOUT.LEFT.UPSTREAM.1FI33511.PV',
 'SPOUT.LEFT.BOS.UPSTREAM.1FI33512.PV',
 'SPOUT.LEFT.BOTTOM.1FI33513.PV',
 'SPOUT.LEFT.TOP.1FI33514.PV',
 'LOR

In [91]:
# re.search에 대한 조사
# re.search에서 여러개의 패턴을 or조건으로 잇기 위해 '' 안에서 |로 이어야 함
#re.search('a|b', 'b') != None

In [96]:
# 확인
#dict_l_r

# Left, Right, Center 구분 딕셔너리에 포함된 컬럼이름을 1차원 리스트로 만듦
# https://programmers.co.kr/learn/courses/4008/lessons/12738
value_l_r = list(dict_l_r.values())
value_l_r_flatten = sum(value_l_r, [])
# 확인
len(value_l_r_flatten)  # 176
len(set(value_l_r_flatten))  # 174 ... 딕셔너리에 중복되어서 들어간 컬럼이 있다는 뜻. 딕셔너리 확인.

174

In [115]:
# 딕셔너리에서 중복된 value 찾기
# 각 키마다 중복된 value가 있는지 조회함
for val in dict_l_r['Center']:
    if val in dict_l_r['Left']:
        print("Center-Left 중복 :", val)
    if val in dict_l_r['Right']:
        print("Center-Right 중복 :", val)
for val in dict_l_r['Right']:
    if val in dict_l_r['Left']:
        print("Center-Right 중복 :", val)

# Center-Right 중복 : E.L.DB.RIGHT.N2.FLOW.1FI33502.PV
# 해당 컬럼은 .L.과 Right를 모두 포함함
# Right에만 속하는 것이 맞으므로 딕셔너리 Left에서 삭제해야 함
# Center -> Right -> Left 판단하도록 수정하여 딕셔너리 재생성

SyntaxError: ignored

In [132]:
dict_a = {'a':[1, 2, 3], 'b':[3,4,5]}
#dir(dict)
#help(dict)

[1, 2, 3]

In [94]:
count=0
for name in col:
    if name not in value_l_r_flatten:
        count += 1
print(count)

667


In [ ]:
# 컬럼집단 하나를 골라 컬럼 간의 상관계수 확인하기
## 만약 이름이 유사한 컬럼 간 종속성이 높다면
## 해당 컬럼 집단에서 하나만 골라 대표로 하고 전체 데이터에 대해 상관계수를 확인할 것

# 컬럼집단명 고르지
col_name_3.keys()

# 'LIF' 컬럼집단의 컬럼명 리스트
col_atm = col_name_3['LIF']
col_atm

# 데이터에서 해당 컬럼만 추출하여 저장하기
df = df_origin[col_atm]
#df

In [ ]:
# 상관계수 데이터프레임을 배열로 저장
corr_arr = df.corr().to_numpy()
## 상관계수는 -1.0~1.0 까지 범위로 나옴.
## 절대값을 취해 0~1.0 범위에서 상관관계가 강한 경우 점수가 높게 나오도록 함 
corr_arr = abs(corr_arr)
#print(corr_arr.shape)    # (26,26)

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 12))
sns.set(font_scale=1.0)
sns.heatmap(corr_arr, cbar=True, square=True, fmt='.2f', annot=True,
           annot_kws={'size':10}, yticklabels=col_atm, xticklabels=col_atm)
plt.show()

In [ ]:
# 모든 컬럼에 대해 상관계수를 구하고 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거

# 모든 컬럼에 대해 상관계수 구하기


# 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거

del_list = []

for i, col in enumerate((corr_arr.columns)):
    for j in range(len(corrMatt.index)):
        if i == j:
            continue
        elif corrMatt.index[i] in del_list:           
            break
        # print(i, j, corrMatt.index[j], np.abs(corrMatt.iloc[j,i]))
        if np.abs(corrMatt.iloc[j,i]) >= 0.9:
            del_list.append(corrMatt.index[j])
            # print(corrMatt.index[j], '-->', corrMatt.index[i])